In [1]:
%load_ext autoreload
%autoreload 2
from rekall import Interval, IntervalSet, IntervalSetMapping, Bounds3D
from rekall.predicates import *
from helpers import *
import time 

[{'num_frames': '1203', 'height': 720, 'width': 1280, 'fps': 29.982553792473208, 'filename': 'cabc30fc-e7726578.mov', 'id': 0}, {'num_frames': '1204', 'height': 720, 'width': 1280, 'fps': 29.99875420455961, 'filename': 'cabc30fc-eb673c5a.mov', 'id': 1}, {'num_frames': '1203', 'height': 720, 'width': 1280, 'fps': 30.006235709831635, 'filename': 'cabc30fc-fd79926f.mov', 'id': 2}, {'num_frames': '1207', 'height': 720, 'width': 1280, 'fps': 30.153641170837325, 'filename': 'cabc9045-1b8282ba.mov', 'id': 3}, {'num_frames': '1210', 'height': 720, 'width': 1280, 'fps': 30.077056922694506, 'filename': 'cabc9045-5a50690f.mov', 'id': 4}]


model loading time:  4 seconds\
model execution time:  1090 seconds = 18 minutes 10 seconds \
bbox file construct time:  5 seconds\
total time:  1140 seconds = 19 minutes

BDD dataset
model loading time:  3.995898962020874
model execution time:  21036.874676465988
video loading time:  2.407963275909424
bbox file construct time:  187.2285144329071
total time:  21806.02480006218

In [2]:
bboxes = get_maskrcnn_bboxes()
visualize_helper([bboxes])

VGridWidget(vgrid_spec={'interval_blocks': [{'interval_sets': [{'name': '0', 'interval_set': [{'bounds': {'t1'…

In [3]:
object_classes = {}
for k in bboxes:
    for interval in bboxes[k].get_intervals():
        if interval['payload']['class'] not in object_classes:
            object_classes[interval['payload']['class']] = 1
        else:
            object_classes[interval['payload']['class']] += 1
print(object_classes)

{'traffic light': 11456, 'car': 44945, 'bicycle': 598, 'baseball bat': 1, 'book': 1, 'person': 3083, 'umbrella': 30, 'bus': 805, 'train': 11, 'truck': 2063, 'stop sign': 37, 'bench': 331, 'motorbike': 635, 'handbag': 32, 'chair': 76, 'parking meter': 2, 'elephant': 1, 'backpack': 8, 'cup': 1, 'pottedplant': 14, 'vase': 2, 'fire hydrant': 14, 'skateboard': 3, 'tvmonitor': 24, 'mouse': 16, 'clock': 1, 'sheep': 12, 'aeroplane': 5, 'suitcase': 20, 'bottle': 1, 'boat': 8, 'surfboard': 6, 'skis': 2, 'kite': 12, 'horse': 1, 'microwave': 3, 'toilet': 1}


In [4]:
bikes = bboxes.filter(lambda interval: interval['payload']['class'] == 'bicycle')
person = bboxes.filter(lambda interval: interval['payload']['class'] == 'person')
car = bboxes.filter(lambda interval: interval['payload']['class'] == 'car')
truck = bboxes.filter(lambda interval: interval['payload']['class'] == 'truck')
stop_sign = bboxes.filter(lambda interval: interval['payload']['class'] == 'stop sign')
traffic_light = bboxes.filter(lambda interval: interval['payload']['class'] == 'traffic light')

In [5]:
visualize_helper([car])

VGridWidget(vgrid_spec={'interval_blocks': [{'interval_sets': [{'name': '0', 'interval_set': [{'bounds': {'t1'…

In [6]:
bike_trackings = bikes.coalesce(
    ('t1', 't2'),
    bounds_merge_op = Bounds3D.span,
    predicate = iou_at_least(0.1),
    epsilon = 5
)

bike_trackings_filtered = bike_trackings.filter_size(
    min_size = 1
)

In [7]:
# car_trackings = car.coalesce(
#     ('t1', 't2'),
#     bounds_merge_op = Bounds3D.span,
#     predicate = iou_at_least(0.5)
# )

# car_trackings_filtered = car_trackings.filter_size(
#     min_size = 5
# )

In [8]:
truck_trackings = truck.coalesce(
    ('t1', 't2'),
    bounds_merge_op = Bounds3D.span,
    predicate = iou_at_least(0.5),
    epsilon=3
)

truck_trackings_filtered = truck_trackings.filter_size(
    min_size = 2
)

In [9]:
truck_before_bike = truck_trackings_filtered.join(
    bike_trackings_filtered,
    predicate = Bounds3D.T(before(min_dist=0, max_dist=20)),
    merge_op = lambda interval1, interval2: Interval(
        interval1['bounds'].span(interval2['bounds'])
    ),
    window = 20.0,
    progress_bar = True
)

100%|██████████| 4/4 [00:00<00:00, 10324.44it/s]


# Truck, followed by bike, with time window = 20.0 s

In [10]:
tik = time.time()
bboxes = get_maskrcnn_bboxes()

car = bboxes.filter(lambda interval: interval['payload']['class'] == 'car')
truck = bboxes.filter(lambda interval: interval['payload']['class'] == 'truck')

car_trackings = car.coalesce(
    ('t1', 't2'),
    bounds_merge_op = Bounds3D.span,
    predicate = iou_at_least(0.5),
    epsilon = 0
)

car_trackings_filtered = car_trackings.filter_size(
    min_size = 1
)

truck_trackings = truck.coalesce(
    ('t1', 't2'),
    bounds_merge_op = Bounds3D.span,
    predicate = iou_at_least(0.5),
    epsilon = 0
)

truck_trackings_filtered = truck_trackings.filter_size(
    min_size = 1
)

truck_before_car = truck_trackings_filtered.join(
    car_trackings_filtered,
    predicate = Bounds3D.T(before(min_dist=0, max_dist=10)),
    merge_op = lambda interval1, interval2: Interval(
        interval1['bounds'].span(interval2['bounds'])
    ),
    window = 10.0,
    progress_bar = True
)
tok = time.time()

print("query time: ", tok - tik)

visualize_helper([truck_before_car, truck_trackings_filtered, car_trackings_filtered])

100%|██████████| 5/5 [00:00<00:00, 1017.15it/s]

query time:  12.639005422592163


VGridWidget(vgrid_spec={'interval_blocks': [{'interval_sets': [{'name': '0', 'interval_set': [{'bounds': {'t1'…

# Count Number of Cars

count(cars + trucks) < 5; count(cars + trucks) > 20; count(cars + trucks) < 5.

Q4: Traffic is light, then heavy, then light again; scenario: bridge is open/close, which affects the traffic condition. 

\begin{equation}
\begin{split}
     &\text{PATTERN } group\_of\_cars\ a; group\_of\_cars\ b; group\_of\_cars\ c \\
    &\text{WHERE } a.count < 5 \text{ AND } b.count > 20 \text{ AND } c.count < 5\\
    & \text{WITHIN } 2 \text{ hours}
\end{split}
\end{equation}

**Note:**
- Doesn't seem to support specifying a time window for a sequence of more than 2 events.
- Operations are either unary of binary.

In [11]:

    
tik = time.time()
bboxes = get_maskrcnn_bboxes()

car = bboxes.filter(lambda interval: interval['payload']['class'] == 'car')
truck = bboxes.filter(lambda interval: interval['payload']['class'] == 'truck')
car_and_truck = car.union(truck)

counts_per_frame = car_and_truck.group_by(
    key=lambda obj: (obj["t1"], obj["t2"]), 
    merge=lambda key, intervalset: Interval(
        Bounds3D(key[0], key[1], 0.3, 0.7, 0.3, 0.7), 
        payload={ 'count': intervalset.size() }
    ) 
)

count_gt_20 = counts_per_frame.filter(lambda intrvl: intrvl['payload']['count'] > 20)
count_lt_5 = counts_per_frame.filter(lambda intrvl: intrvl['payload']['count'] < 5)

ls5_gt20 = count_lt_5.join(
    count_gt_20,
    predicate = Bounds3D.T(before(min_dist=0, max_dist=20)),
    merge_op = lambda interval1, interval2: Interval(
        interval1['bounds'].span(interval2['bounds'])
    ),
    window = 20.0,
    progress_bar = True
)

ls5_gt20_ls5 = ls5_gt20.join(
    count_lt_5,
    predicate = Bounds3D.T(before(min_dist=0, max_dist=20)),
    merge_op = lambda interval1, interval2: Interval(
        interval1['bounds'].span(interval2['bounds'])
    ),
    window = 20.0,
    progress_bar = True
)

tok = time.time()

print("query time: ", tok - tik)

visualize_helper([ls5_gt20_ls5, count_lt_5, count_gt_20, car, truck])

100%|██████████| 5/5 [00:00<00:00, 10.24it/s]

query time:  3.7051639556884766


VGridWidget(vgrid_spec={'interval_blocks': [], 'settings': {'spinner_dev_mode': False, 'key_mode': 'Jupyter', …

In [12]:
print(ls5_gt20_ls5)

{}
